In [1]:
import sys
import os

sys.path.append(r'c:/users/george.crowther/documents/python/projects')

from optasense.plotting.well_plot import Well_Plot
from optasense.optaframe import optaframe

import h5py as h5
import numpy as np
import pandas as pd

import bokeh.plotting as bk
from bokeh.palettes import viridis
from bokeh.charts.builders import heatmap_builder
from bokeh.models import DatetimeTickFormatter, Range1d, DatetimeAxis, LinearAxis

import time
from time import mktime
from datetime import datetime

def non_uniform_array_image(frame, zmin, zmax, width = 800, height = 800):
    """
    Fundamental plotting method for 2D data in a Pandas Dataframe.
    Args:
        frame: Pandas Dataframe with timestamped index and floating point channel locations as column headers
        zmin: Colormap min value
        zmax: Colormap max value
    Returns:
        p: Figure image containing colormap
        cbar: Figure based colorbar for 'p'"""
    
    y_values = frame.columns
    y_mids = np.vstack((frame.columns[:-1], frame.columns[1:])).mean(axis=0)
    y_heights = np.diff(y_mids)
    y_heights = np.hstack(([2*(y_mids[0] - y_values[0])],
                               y_heights,
                               2*(y_values[-1] - y_mids[-1])))
    rows = len(y_values)
    
    x_values = frame.index.asi8
    x_mids = np.vstack((x_values[:-1], x_values[1:])).mean(axis=0)
    x_widths = np.diff(x_mids)
    x_widths = np.hstack(([2*(x_mids[0] - x_values[0])],
                               x_widths,
                               2*(x_values[-1] - x_mids[-1])))
    columns = len(x_values)
    
    std = frame.std().std()
    zmin = zmin#frame.mean().mean() - 1*std
    zmax = zmax#frame.mean().mean() + 4*std
    
    proj_array = np.ones(frame.shape)
    
    mapped_data, cbar = colormap_data(frame, zmin, zmax, width)
    
    x = np.reshape((x_values * proj_array.T).T, -1)
    y = np.reshape(y_values.values * proj_array, -1)
    widths = np.reshape((x_widths * proj_array.T).T, -1)
    heights = np.reshape(y_heights * proj_array, -1)
    colors = np.reshape(mapped_data, -1)
           
    p = bk.figure(width = width, height = height, x_range = [x_values[0], x_values[-1]], 
                  y_range = [y_values[-1], y_values[0]]) 
    p.rect(x = x, y = y, width = widths, height = heights, color = colors)
    
    dt_format = DatetimeTickFormatter(hours=["%d %B %Y %H:%M:%S"], days=["%d %B %Y"], months=["%d %B %Y"], years=["%d %B %Y"])
    
    p.xaxis.visible = False
    p.extra_x_ranges = {'Time': Range1d(frame.index[0], frame.index[-1])}
    p.add_layout(DatetimeAxis(x_range_name = 'Time'), 'below')
    p.xaxis[1].formatter = dt_format
    p.xaxis[1].major_label_orientation = 3.14159/4
    
    return p, cbar

def color_bar(cmap, bins, width):
    p = bk.figure(height = 50, width = width, x_range = [bins[0], bins[-1]])
    p.yaxis.visible = False
    p.toolbar_location = None
    p.toolbar.logo = None
    cmap = cmap
    
    def m(x):
        return cmap[x]
    
    f = np.vectorize(m)
    
    x = bins
    y = [0] * len(bins)
    widths = [np.mean(np.diff(x))] * len(bins)
    heights = [1] * len(bins)
    colors = f(range(len(bins)))
    
    p.rect(x = x, y = y, width = widths, height = heights, color = colors)
    
    return p

def colormap_data(frame, zmin, zmax, width):
    cmap = viridis(256)
    
    bins = np.linspace(zmin, zmax, len(cmap) - 1)
    
    cbar = color_bar(cmap, bins, width)
    
    binned = np.digitize(frame, bins)
    
    def m(x):
        return cmap[x]
    
    f = np.vectorize(m)
    
    return f(binned), cbar

def combined_plot(well_frame, frame_list, height, width_list, titles, cmap = 'Viridis'):
    
    well = Well_Plot(well_frame, height = height, width = 200)
    well.create_plot()
    well.figure.min_border_left = 0
    
    frame_plots = [non_uniform_array_image(frame, frame.min().min(), frame.max().max(), width = width_list[i], height = height) for i, frame in enumerate(frame_list)]
    
    frame_plots[0][0].y_range = well.figure.y_range
    frame_plots[0][0].yaxis.visible = False
    frame_plots[0][0].xaxis[1].axis_label = titles[0]
    
    for i, plot in enumerate(frame_plots[1:]):
        plot[0].x_range = frame_plots[0][0].x_range
        plot[0].y_range = well.figure.y_range
        plot[0].yaxis.visible = False
        plot[0].xaxis[1].axis_label = titles[i+1]
    
    blank_plot = bk.figure(height = frame_plots[0][1].plot_height, width = well.figure.plot_width)
    
    upper_row = [blank_plot]
    lower_row = [well.figure]
    
    [upper_row.append(plot[1]) for plot in frame_plots]
    [lower_row.append(plot[0]) for plot in frame_plots]
    
    grid = [upper_row, lower_row]
    
    return grid

def combined_plot_B(well_frame, frame_list, row_3_curve_list, row_4_curve_list, height, width_list, titles, cmap = 'Viridis'):
    
    #Clear current document, plot sizes are typically large and can cause errors
    bk.curdoc().clear()
    
    #Create combined plot, including wellframe, yields a list of lists, each containing the figures pertinent to the first two rows of the grid.
    print('Creating grid')
    grid = combined_plot(well_frame, frame_list, height, width_list, titles, cmap)
    grid[-1][-1].xaxis.visible = False
    
    # Blank lists for control rows 3 and 4 (each row to start with a blank plot)
    row_3 = [bk.figure(height = 200, width = grid[0][0].plot_width)]
    row_4 = [bk.figure(height = 200, width = grid[0][0].plot_width)]
    
    # Iterate over second row of grid to create row_3 and row_4 plots
    print('Creating surface plots')
    for i, fig in enumerate(grid[1][1:]):
        row_3.append(bk.figure(height = 175, width = fig.plot_width, x_range = fig.extra_x_ranges['Time'], x_axis_type = 'datetime'))
        row_4.append(bk.figure(height = 175, width = fig.plot_width, x_range = fig.extra_x_ranges['Time'], x_axis_type = 'datetime'))
        
        # Define colormap and create index for use
        line_cmap = viridis(len(row_3_curve_list[i].columns))
        k = 0
        
        row_3[-1].yaxis.visible = False
        row_3[-1].min_border_left = 5
        row_3[-1].min_border_bottom = 5
        row_3[-1].xaxis.visible = False
        
        for j, col in row_3_curve_list[i].iteritems():
            row_3[-1].extra_y_ranges[j] = Range1d(col.min(), col.max())
            row_3[-1].add_layout(LinearAxis(y_range_name = j), 'right')
            row_3[-1].yaxis[-1].axis_line_color = line_cmap[k]
            row_3[-1].yaxis[-1].axis_label = j
            row_3[-1].yaxis[-1].axis_label_text_color = line_cmap[k]
            row_3[-1].yaxis[-1].major_label_text_color = line_cmap[k]
            row_3[-1].yaxis[-1].major_tick_line_color = line_cmap[k]
            row_3[-1].yaxis[-1].minor_tick_line_color = line_cmap[k]
            
            row_3[-1].line(col.index, col, color = line_cmap[k], y_range_name = j)
            k += 1
        
        line_cmap = viridis(len(row_4_curve_list[i].columns))
        k = 0
        
        row_4[-1].yaxis.visible = False
        row_4[-1].min_border_left = 5
        row_4[-1].min_border_top = 5
        
        for j, col in row_4_curve_list[i].iteritems():
            row_4[-1].extra_y_ranges[j] = Range1d(col.min(), col.max())
            row_4[-1].add_layout(LinearAxis(y_range_name = j), 'right')
            row_4[-1].yaxis[-1].axis_line_color = line_cmap[k]
            row_4[-1].yaxis[-1].axis_label = j
            row_4[-1].yaxis[-1].axis_label_text_color = line_cmap[k]
            row_4[-1].yaxis[-1].major_label_text_color = line_cmap[k]
            row_4[-1].yaxis[-1].major_tick_line_color = line_cmap[k]
            row_4[-1].yaxis[-1].minor_tick_line_color = line_cmap[k]
            
            row_4[-1].line(col.index, col, color = line_cmap[k], y_range_name = j)
            k += 1
            
    grid.append(row_3)
    grid.append(row_4)
    
    return grid

C:\Users\george.crowther\AppData\Local\Continuum\Anaconda3_4\envs\rice_energy\lib\site-packages\bokeh\util\deprecation.py:33: BokehDeprecationWarning: DatetimeTickFormatter.formats was deprecated in Bokeh 0.12.3 and will be removed, use individual DatetimeTickFormatter fields instead.
  warn(message)


In [2]:
FPATH = r'C:\Users\george.crowther\Documents\Python\Projects\rice_energy\data'

In [3]:
file_paths = []
for directory, sub_dir, filename in os.walk(FPATH):
    for file in filename:
        file_paths.append(os.path.join(directory, file))

In [4]:
output_path = r'C:\Users\george.crowther\Documents\Python\Projects\rice_energy\html_images'
failures = []
for file in file_paths:
    dsets = [optaframe.OptaFrame([file], loader = 'FBE_H5', band = n) for n in range(5)]
    
    for i, band in enumerate(dsets):
        dset = 10*np.log10(band.resample('10S').mean())
        bk.curdoc().clear()
        
        fname = "{0} - Band {1}".format(file.split('\\')[-2], i)
        
        html_path = os.path.join(output_path, "{0} - B.html".format(fname))
        
        if os.path.exists(html_path):
            continue
        else:
            try:
                bk.output_file(os.path.join(output_path, "{0} - B.html".format(fname)), title = fname)

                p, cbar = non_uniform_array_image(dset, dset.min().min(), dset.max().max(), width = 1500)
                grid = bk.gridplot([[cbar], [p]])
                bk.show(grid)
            except:
                failures.append(fname)

In [5]:
file_paths

['C:\\Users\\george.crowther\\Documents\\Python\\Projects\\rice_energy\\data\\Stage 01 Actual\\FBE_20160206T112115.h5',
 'C:\\Users\\george.crowther\\Documents\\Python\\Projects\\rice_energy\\data\\Stage 01 Offset\\FBE_20160206T110732.h5',
 'C:\\Users\\george.crowther\\Documents\\Python\\Projects\\rice_energy\\data\\Stage 02\\FBE_20160206T203653.h5',
 'C:\\Users\\george.crowther\\Documents\\Python\\Projects\\rice_energy\\data\\Stage 07\\FBE_20160219T125540.h5',
 'C:\\Users\\george.crowther\\Documents\\Python\\Projects\\rice_energy\\data\\Stage 08\\FBE_20160220T140240.h5',
 'C:\\Users\\george.crowther\\Documents\\Python\\Projects\\rice_energy\\data\\Stage 09\\FBE_20160221T021510.h5',
 'C:\\Users\\george.crowther\\Documents\\Python\\Projects\\rice_energy\\data\\Stage 10\\FBE_20160221T123303.h5',
 'C:\\Users\\george.crowther\\Documents\\Python\\Projects\\rice_energy\\data\\Stage 11\\FBE_20160221T233850.h5',
 'C:\\Users\\george.crowther\\Documents\\Python\\Projects\\rice_energy\\data\\Stag